In [13]:
import time
import pygame
import math
import random

class Section:
    def __init__(self,p):
        self.p1 = p[0]
        self.p2 = p[1]
        self.p3 = p[2]
        self.p4 = p[3]
    def getArea(self):
        return (1)*(abs(self.p1[0]-self.p4[0]) + abs(self.p2[0]-self.p3[0]))/2
    def getSlopeAndIntercept(self,p1,p2):
        x1 = p1[0]
        y1 = p1[1]
        x2 = p2[0]
        y2 = p2[1]
        slope = (y1-y2)/(x1-x2)
        intercept = y1 - (slope * x1)
        return [slope,intercept]
    def getFuzzyValue(self,x):
        if x < self.p1[0] or x > self.p4[0]:
            return 0
        elif x >= self.p1[0] and x <= self.p2[0] : 
            slopeAndIntercept = self.getSlopeAndIntercept(self.p1,self.p2)
            return slopeAndIntercept[0]*x + slopeAndIntercept[1]; 
        elif x >= self.p2[0] and x <= self.p3[0] : 
            return 1
        elif x >= self.p3[0] and x <= self.p4[0] : 
            slopeAndIntercept = self.getSlopeAndIntercept(self.p3,self.p4)
            return slopeAndIntercept[0]*x + slopeAndIntercept[1]; 


In [16]:
NEGATIVE_MED_P = [[-1000,0],[-800,1],[-200,1],[-5,0]]
NEGATIVE_SMALL_P = [[-100,0],[-80,1],[-20,1],[0,0]]
ZERO_P = [[-10,0],[0,1],[0,1],[10,0]]
POSITIVE_SMALL_P = [[0,0],[20,1],[80,1],[100,0]]
POSITIVE_MED_P = []

CURR_SCALE = 150
ANGULAR_VEL_SCALE = 300
ANGLE_LIMIT = 5
CURR_LIMIT = 200
ANGULAR_ACC_SCALE = 100

angle = 1
angular_vel = 1
angular_acc = 0
angular_vel_scale = 1

negative_med = Section(NEGATIVE_MED_P)
negative_small = Section(NEGATIVE_SMALL_P)
zero = Section(ZERO_P)
positive_small = Section(POSITIVE_SMALL_P)

curr = 0
i=0

In [17]:
pygame.init()

SIZE = 640, 480
screen = pygame.display.set_mode(SIZE)
FPSCLOCK = pygame.time.Clock()
    
startpoint = pygame.math.Vector2(320, 240)
endpoint = pygame.math.Vector2(0, -170)


done = False
while not done:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True
    
    effective_angular_vel = angular_vel * angular_vel_scale
    u_zero_angle = zero.getFuzzyValue(angle)
    u_zero_angular_vel = zero.getFuzzyValue(effective_angular_vel)
    u_positive_small_angle = positive_small.getFuzzyValue(angle)
    u_positive_small_angular_vel = positive_small.getFuzzyValue(effective_angular_vel)
    #u_positive_med_angle = positive_med.getFuzzyValue(angle)
    #u_positive_med_anglular_vel = positive_med.getFuzzyValue(effective_angular_vel)
    #u_negative_small_angle = negative_small.getFuzzyValue(angle)
    #u_negative_small_angular_vel = negative_small.getFuzzyValue(effective_angular_vel)
    #u_negative_med_angle = negative_med.getFuzzyValue(angle)
    #u_negative_med_anglular_vel = negative_med.getFuzzyValue(effective_angular_vel)
    
    if angle < 0:
        u_zero_angle = zero.getFuzzyValue(-angle)
        u_positive_small_angle = positive_small.getFuzzyValue(-angle)
        #u_positive_med_angle = positive_med.getFuzzyValue(angle)
        #u_positive_med_anglular_vel = positive_med.getFuzzyValue(effective_angular_vel)
     #   u_negative_small_angle = negative_small.getFuzzyValue(-angle)
    if angular_vel < 0:
        u_zero_angular_vel = zero.getFuzzyValue(-effective_angular_vel)
        u_positive_small_angular_vel = positive_small.getFuzzyValue(-effective_angular_vel)
        
    print("current - ",curr)  
    print("angle - ",angle)
    print("angular_vel - ",angular_vel)
    print("angular_acc - ",angular_acc)
    if angle == 0 and angular_vel == 0 : 
        curr = 0 
    elif angle == 0 and angular_vel > 0 : #-ve small
        print("")
    elif angle == 0 and angular_vel < 0 : #+ve small
        print("")
        
    
    elif angle > 0 and angular_vel == 0 : #-ve small
        print("")
    elif angle > 0 and angular_vel > 0 : #-ve med
        zero_trap_height = min(u_zero_angle,u_zero_angular_vel)
        neg_small_trap_height1 = min(u_zero_angle,u_positive_small_angular_vel)
        neg_small_trap_height2 = min(u_positive_small_angle,u_zero_angular_vel)
        neg_med_trap_height = min(u_positive_small_angle,u_positive_small_angular_vel)
        
        zero_trap_area = zero_trap_height * zero.getArea()
        neg_small_trap_height1_area = neg_small_trap_height1 * negative_small.getArea()
        neg_small_trap_height2_area = neg_small_trap_height2 * negative_small.getArea()
        neg_med_trap_area = neg_med_trap_height * negative_med.getArea()
        
        area_sum = zero_trap_area + neg_small_trap_height1_area + neg_small_trap_height2_area  + neg_med_trap_area
        
        centroid_area_zero = ((zero.p1[0] + zero.p4[0])/2)*zero_trap_area
        centroid_area_neg_small1 = ((negative_small.p1[0] + negative_small.p4[0])/2)*neg_small_trap_height1_area
        centroid_area_negsmall2 = ((negative_small.p1[0] + negative_small.p4[0])/2)*neg_small_trap_height2_area
        centroid_area_zero = ((negative_med.p1[0] + negative_med.p4[0])/2)* neg_med_trap_area 
        
        curr = (centroid_area_zero + centroid_area_neg_small1 + centroid_area_negsmall2 + centroid_area_zero)/area_sum
        
        
        
        
    elif angle > 0 and angular_vel < 0 : 
        curr = 0
    
    
    elif angle < 0 and angular_vel == 0 : #+ve small
        print("")
    elif angle < 0 and angular_vel > 0 : 
        curr = 0
    elif angle < 0 and angular_vel < 0 : #+ve med
        
        zero_trap_height = min(u_zero_angle,u_zero_angular_vel)
        neg_small_trap_height1 = min(u_zero_angle,u_positive_small_angular_vel)
        neg_small_trap_height2 = min(u_positive_small_angle,u_zero_angular_vel)
        neg_med_trap_height = min(u_positive_small_angle,u_positive_small_angular_vel)
        
        
        zero_trap_area = zero_trap_height * zero.getArea()
        neg_small_trap_height1_area = neg_small_trap_height1 * negative_small.getArea()
        neg_small_trap_height2_area = neg_small_trap_height2 * negative_small.getArea()
        neg_med_trap_area = neg_med_trap_height * negative_med.getArea()
        
        area_sum = zero_trap_area + neg_small_trap_height1_area + neg_small_trap_height2_area  + neg_med_trap_area
        
        centroid_area_zero = ((zero.p1[0] + zero.p4[0])/2)*zero_trap_area
        centroid_area_neg_small1 = ((negative_small.p1[0] + negative_small.p4[0])/2)*neg_small_trap_height1_area
        centroid_area_negsmall2 = ((negative_small.p1[0] + negative_small.p4[0])/2)*neg_small_trap_height2_area
        centroid_area_zero = ((negative_med.p1[0] + negative_med.p4[0])/2)* neg_med_trap_area 
        
        curr = (centroid_area_zero + centroid_area_neg_small1 + centroid_area_negsmall2 + centroid_area_zero)/area_sum
        curr = -curr
        
    if curr > CURR_LIMIT :
        curr = CURR_LIMIT
    if curr < -CURR_LIMIT :
        curr = -CURR_LIMIT
    if angle > ANGLE_LIMIT : 
        angle = ANGLE_LIMIT
        angular_vel = -angular_vel
        
    if angle < -ANGLE_LIMIT:
        angle = -ANGLE_LIMIT
        angular_vel = -angular_vel
    angular_acc = angular_acc + curr/CURR_SCALE
    angular_vel = angular_vel + angular_acc/ANGULAR_ACC_SCALE
    
    prev_angle = angle
    if angular_vel > 0 :
        angle = angle + angular_vel*angular_vel/ANGULAR_VEL_SCALE
    else :
        angle = angle - angular_vel*angular_vel/ANGULAR_VEL_SCALE
        
    if prev_angle*angle < 0:
        angular_acc = 0
   
            
    current_endpoint = startpoint + endpoint.rotate(angle)

    screen.fill((0, 0, 0))
    pygame.draw.line(
        screen, pygame.Color("red"), startpoint, current_endpoint, 2)

    pygame.display.flip()  
    FPSCLOCK.tick(60)

pygame.quit()

current -  0
angle -  1
angular_vel -  1
angular_acc -  0
current -  -200
angle -  1.003245037037037
angular_vel -  0.9866666666666667
angular_acc -  -1.3333333333333333
current -  -200
angle -  1.006317037037037
angular_vel -  0.9600000000000001
angular_acc -  -2.6666666666666665
current -  -200
angle -  1.0091383703703702
angular_vel -  0.92
angular_acc -  -4.0
current -  -200
angle -  1.011642074074074
angular_vel -  0.8666666666666667
angular_acc -  -5.333333333333333
current -  -200
angle -  1.0137754074074072
angular_vel -  0.8
angular_acc -  -6.666666666666666
current -  -200
angle -  1.0155034074074072
angular_vel -  0.7200000000000001
angular_acc -  -7.999999999999999
current -  -200
angle -  1.0168124444444442
angular_vel -  0.6266666666666667
angular_acc -  -9.333333333333332
current -  -200
angle -  1.0177137777777776
angular_vel -  0.52
angular_acc -  -10.666666666666666
current -  -200
angle -  1.0182471111111109
angular_vel -  0.4
angular_acc -  -12.0
current -  -200
ang

current -  0
angle -  -1.1977844240092286
angular_vel -  4.094888059025531
angular_acc -  37.40746912696804
current -  0
angle -  -1.1312123304641415
angular_vel -  4.468962750295212
angular_acc -  37.40746912696804
current -  0
angle -  -1.05302895826281
angular_vel -  4.843037441564892
angular_acc -  37.40746912696804
current -  0
angle -  -0.962301428240911
angular_vel -  5.217112132834572
angular_acc -  37.40746912696804
current -  0
angle -  -0.8580968612341211
angular_vel -  5.591186824104252
angular_acc -  37.40746912696804
current -  0
angle -  -0.7394823780781168
angular_vel -  5.965261515373932
angular_acc -  37.40746912696804
current -  0
angle -  -0.6055250996085747
angular_vel -  6.339336206643612
angular_acc -  37.40746912696804
current -  0
angle -  -0.45529214666117157
angular_vel -  6.713410897913292
angular_acc -  37.40746912696804
current -  0
angle -  -0.2878506400715839
angular_vel -  7.087485589182972
angular_acc -  37.40746912696804
current -  0
angle -  -0.10226

current -  200
angle -  -5.2556319154026205
angular_vel -  -10.194269887933956
angular_acc -  16.81143490074993
current -  200
angle -  -4.641148283006303
angular_vel -  10.375717570274789
angular_acc -  18.144768234083262
current -  0
angle -  -4.269635822436188
angular_vel -  10.557165252615622
angular_acc -  18.144768234083262
current -  0
angle -  -3.8852431298801404
angular_vel -  10.738612934956455
angular_acc -  18.144768234083262
current -  0
angle -  -3.48775071692865
angular_vel -  10.920060617297288
angular_acc -  18.144768234083262
current -  0
angle -  -3.076939095172203
angular_vel -  11.101508299638121
angular_acc -  18.144768234083262
current -  0
angle -  -2.6525887762012874
angular_vel -  11.282955981978954
angular_acc -  18.144768234083262
current -  0
angle -  -2.2144802716063907
angular_vel -  11.464403664319788
angular_acc -  18.144768234083262
current -  0
angle -  -1.7623940929780006
angular_vel -  11.64585134666062
angular_acc -  18.144768234083262
current -  0

current -  -200
angle -  1.861458209990952
angular_vel -  14.624975409357285
angular_acc -  -2.6666666666666665
current -  -200
angle -  2.570529902296141
angular_vel -  14.584975409357286
angular_acc -  -4.0
current -  -200
angle -  3.2744253070483733
angular_vel -  14.531642076023953
angular_acc -  -5.333333333333333
current -  -200
angle -  3.971877019026077
angular_vel -  14.464975409357287
angular_acc -  -6.666666666666666
current -  -200
angle -  4.661635410785456
angular_vel -  14.384975409357287
angular_acc -  -7.999999999999999
current -  -200
angle -  5.34247218821605
angular_vel -  14.291642076023953
angular_acc -  -9.333333333333332
current -  -200
angle -  4.308962351167196
angular_vel -  -14.39830874269062
angular_acc -  -10.666666666666666
current -  0
angle -  3.6076479790803306
angular_vel -  -14.504975409357288
angular_acc -  -10.666666666666666
current -  0
angle -  2.895981031887552
angular_vel -  -14.611642076023955
angular_acc -  -10.666666666666666
current -  0
a

current -  -200
angle -  4.933865246931715
angular_vel -  16.424975409357288
angular_acc -  -6.666666666666666
current -  -200
angle -  5.824392650706697
angular_vel -  16.34497540935729
angular_acc -  -7.999999999999999
current -  -200
angle -  4.099273352266604
angular_vel -  -16.438308742690623
angular_acc -  -9.333333333333332
current -  0
angle -  3.188289386500718
angular_vel -  -16.531642076023957
angular_acc -  -9.333333333333332
current -  0
angle -  2.2669900286282694
angular_vel -  -16.62497540935729
angular_acc -  -9.333333333333332
current -  0
angle -  1.3353172045751835
angular_vel -  -16.718308742690624
angular_acc -  -9.333333333333332
current -  0
angle -  0.39321284026738634
angular_vel -  -16.811642076023958
angular_acc -  -9.333333333333332
current -  0
angle -  -0.5593811383691961
angular_vel -  -16.90497540935729
angular_acc -  0
current -  200
angle -  -1.5104730451175397
angular_vel -  -16.89164207602396
angular_acc -  1.3333333333333333
current -  200
angle - 

...

angular_vel -  -23.95830874269072
angular_acc -  2.6666666666666665
current -  200
angle -  -6.471521090871075
angular_vel -  -23.91830874269072
angular_acc -  4.0
current -  200
angle -  -3.0845345872633105
angular_vel -  23.971642076024054
angular_acc -  5.333333333333333
current -  0
angle -  -1.1605364425292197
angular_vel -  24.02497540935739
angular_acc -  5.333333333333333
current -  0
angle -  0.7720133971652352
angular_vel -  24.078308742690723
angular_acc -  0
current -  -200
angle -  2.702423535341822
angular_vel -  24.06497540935739
angular_acc -  -1.3333333333333333
current -  -200
angle -  4.628557826038226
angular_vel -  24.038308742690724
angular_acc -  -2.6666666666666665
current -  -200
angle -  6.548287234403246
angular_vel -  23.998308742690725
angular_acc -  -4.0
current -  -200
angle -  3.071728378156097
angular_vel -  -24.05164207602406
angular_acc -  -5.333333333333333
current -  0
angle -  1.1348955798703482
angular_vel -  -24.104975409357394
angular_acc -  -5.

current -  0
angle -  -1.3697329485804657
angular_vel -  -25.27830874269074
angular_acc -  0
current -  200
angle -  -3.4974628900319145
angular_vel -  -25.264975409357408
angular_acc -  1.3333333333333333
current -  200
angle -  -5.620703650669848
angular_vel -  -25.23830874269074
angular_acc -  2.6666666666666665
current -  200
angle -  -2.8700236903640164
angular_vel -  25.27830874269074
angular_acc -  4.0
current -  0
angle -  -0.7333011650633154
angular_vel -  25.31830874269074
angular_acc -  4.0
current -  0
angle -  1.4101782425687697
angular_vel -  25.35830874269074
angular_acc -  0
current -  -200
angle -  3.5514041709052084
angular_vel -  25.344975409357406
angular_acc -  -1.3333333333333333
current -  -200
angle -  5.688126696205909
angular_vel -  25.31830874269074
angular_acc -  -2.6666666666666665
current -  -200
angle -  2.856520592367915
angular_vel -  -25.35830874269074
angular_acc -  -4.0
current -  0
angle -  0.7062736357377788
angular_vel -  -25.398308742690737
angul

current -  200
angle -  -2.6488541224263935
angular_vel -  26.558308742690713
angular_acc -  4.0
current -  0
angle -  -0.29062069585473616
angular_vel -  26.598308742690712
angular_acc -  4.0
current -  0
angle -  2.0747109463816384
angular_vel -  26.63830874269071
angular_acc -  0
current -  -200
angle -  4.43767533154459
angular_vel -  26.62497540935738
angular_acc -  -1.3333333333333333
current -  -200
angle -  6.795908758116247
angular_vel -  26.598308742690712
angular_acc -  -2.6666666666666665
current -  -200
angle -  2.6346683577636254
angular_vel -  -26.63830874269071
angular_acc -  -4.0
current -  0
angle -  0.26222783319586673
angular_vel -  -26.67830874269071
angular_acc -  -4.0
current -  0
angle -  -2.1173322403699424
angular_vel -  -26.71830874269071
angular_acc -  0
current -  200
angle -  -4.494517945751216
angular_vel -  -26.704975409357377
angular_acc -  1.3333333333333333
current -  200
angle -  -6.866958470318975
angular_vel -  -26.67830874269071
angular_acc -  2.6

current -  -109.10294635389009
angle -  2.697692604890828
angular_vel -  27.75103521293377
angular_acc -  -0.7273529756926006
current -  -200
angle -  5.260948126443511
angular_vel -  27.73042834984351
angular_acc -  -2.0606863090259337
current -  -200
angle -  2.4304661307568463
angular_vel -  -27.764368546267104
angular_acc -  -3.3940196423592672
current -  0
angle -  -0.14535376575633396
angular_vel -  -27.798308742690697
angular_acc -  0
current -  115.40182141587145
angle -  -2.7197480926953626
angular_vel -  -27.79061528792964
angular_acc -  0.7693454761058097
current -  200
angle -  -5.290248244195969
angular_vel -  -27.769588499835248
angular_acc -  2.102678809439143
current -  200
angle -  -2.4231348035538964
angular_vel -  27.803948621262972
angular_acc -  3.4360121427724764
current -  0
angle -  0.1601033086240009
angular_vel -  27.838308742690696
angular_acc -  0
current -  -121.64040126701371
angle -  2.741836634209388
angular_vel -  27.830199382606228
angular_acc -  -0.81

angular_acc -  -4.0
current -  0
angle -  -0.444800870617009
angular_vel -  -28.598308742690694
angular_acc -  0
current -  200
angle -  -3.168470267795559
angular_vel -  -28.584975409357362
angular_acc -  1.3333333333333333
current -  200
angle -  -5.887060261938371
angular_vel -  -28.558308742690695
angular_acc -  2.6666666666666665
current -  200
angle -  -2.273789123525803
angular_vel -  28.598308742690694
angular_acc -  4.0
current -  0
angle -  0.4600533019464441
angular_vel -  28.638308742690693
angular_acc -  0
current -  -200
angle -  3.1913506925674895
angular_vel -  28.62497540935736
angular_acc -  -1.3333333333333333
current -  -200
angle -  5.917561569041686
angular_vel -  28.598308742690694
angular_acc -  -2.6666666666666665
current -  -200
angle -  2.2661575745277527
angular_vel -  -28.638308742690693
angular_acc -  -4.0
current -  0
angle -  -0.475327066609212
angular_vel -  -28.678308742690692
angular_acc -  0
current -  200
angle -  -3.2142631173394194
angular_vel -  

current -  -8.924019607843139
angle -  1.0153346677059305
angular_vel -  0.5537990196078431
angular_acc -  -0.44620098039215694
current -  -7.663407476977831
angle -  1.016790374443607
angular_vel -  0.17062864575895154
angular_acc -  -0.3831703738488915
current -  -5.935809316564023
angle -  1.0159945342014478
angular_vel -  -0.1261618200692496
angular_acc -  -0.2967904658282011
current -  0
angle -  1.0151986939592885
angular_vel -  -0.1261618200692496
angular_acc -  0.0
current -  0
angle -  1.0144028537171292
angular_vel -  -0.1261618200692496
angular_acc -  0.0
current -  0
angle -  1.01360701347497
angular_vel -  -0.1261618200692496
angular_acc -  0.0
current -  0
angle -  1.0128111732328107
angular_vel -  -0.1261618200692496
angular_acc -  0.0
current -  0
angle -  1.0120153329906514
angular_vel -  -0.1261618200692496
angular_acc -  0.0
current -  0
angle -  1.011219492748492
angular_vel -  -0.1261618200692496
angular_acc -  0.0
current -  0
angle -  1.0104236525063328
angular_v

KeyboardInterrupt: 